In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import tqdm
import time 

def get_data(url):
    page = 0
    base_url = url
    articles = []  

    # create a csv file to store data 
    csv_file = "data_dantri.csv"
    with open(csv_file, 'w', encoding='utf-8') as file:
        # write the header of the csv file
        file.write("link,title,time,content\n")

    while True:
        try:
            url = base_url + str(page) + ".htm"

            page += 1
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            target = soup.find('div', class_='article list')

            # break code if there is no more article
            if target == None:
                break

            for i in tqdm.tqdm(target.find_all('h3')):
                article_data = {
                    'link': "https://dantri.com.vn" + i.find('a')['href'],
                    'title': i.text.strip(),
                    'time' : [],
                    'content': []
                }
                response = requests.get(article_data['link'])
                article_soup = BeautifulSoup(response.text, 'html.parser')
                article_data['time'] = article_soup.find('time', class_='author-time').text.strip().split(" ")[2]
                article_content = article_soup.find('div', class_='singular-content')   
                if article_content:
                    for p in article_content.find_all('p'):
                        article_data['content'].append(p.text.strip())
                    
                    article_data['content'] = " ".join(article_data['content'])    

                articles.append(article_data)
                
                # open file csv and write data to it
                with open(csv_file, 'a', encoding='utf-8') as file:
                    file.write(f"{article_data['link']},{article_data['title']},{article_data['time']},{article_data['content']}\n")
        
                time.sleep(0.05)

        except Exception as e:
            print(e)
            continue

def main():
    url = "https://dantri.com.vn/kinh-doanh/chung-khoan/trang-"
    get_data(url)

if __name__ == '__main__':
    main()